In [ ]:
!pip install mlflow boto3 -q

     |████████████████████████████████| 17.9 MB 4.9 MB/s 
     |████████████████████████████████| 132 kB 70.2 MB/s 
     |████████████████████████████████| 81 kB 12.0 MB/s 
     |████████████████████████████████| 79 kB 10.5 MB/s 
     |████████████████████████████████| 209 kB 66.9 MB/s 
     |████████████████████████████████| 146 kB 56.2 MB/s 
     |████████████████████████████████| 181 kB 71.8 MB/s 
     |████████████████████████████████| 596 kB 70.5 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 9.0 MB 48.2 MB/s 
     |████████████████████████████████| 79 kB 10.7 MB/s 
     |████████████████████████████████| 127 kB 77.5 MB/s 
     |████████████████████████████████| 78 kB 8.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 req

In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["MLFLOW_TRACKING_URI"] = "http://:5000"

In [ ]:
import pandas as pd

df = pd.read_csv("train-1000.csv")

In [ ]:
import boto3

s3_client = boto3.client('s3')
os.makedirs("/content/train_data/with_mask", exist_ok=True)
os.makedirs("/content/train_data/without_mask", exist_ok=True)

def download_files(df, bucket_name="jeongmin-mask-detection-train-dataset", s3_prefix="mask_images", local_prefix="/content/train_data"):
    for idx in range(len(df)):
        class_name = df["class"][idx]
        file_name = df["file_name"][idx]

        s3_path = f"{s3_prefix}/{class_name}/{file_name}"
        local_path = f"{local_prefix}/{class_name}/{file_name}"

        s3_client.download_file(bucket_name, s3_path, local_path)

In [ ]:
download_files(df=df)

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras import Sequential
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Load train and test set
train_dir = '/content/train_data'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, zoom_range=0.2,shear_range=0.2)
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(128,128),
    class_mode='categorical',
    batch_size=32
)

Found 1000 images belonging to 2 classes.


In [ ]:
import mlflow 

mlflow.keras.autolog()

/usr/local/lib/python3.7/dist-packages/mlflow/utils/autologging_utils/__init__.py:416: FutureWarning: Autologging support for keras >= 2.6.0 has been deprecated and will be removed in a future MLflow release. Use `mlflow.tensorflow.autolog()` instead.
  return _autolog(*args, **kwargs)


In [ ]:
vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(128,128,3))

for layer in vgg19.layers:
    layer.trainable = False
    
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_1 (Dense)             (None, 2)                 16386     
                                                                 
Total params: 20,040,770
Trainable params: 16,386
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics ="accuracy")

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=len(train_generator)//32,
                              epochs=20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until
2022/07/04 18:04:29 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '634d260a11aa4e748d3f6aabdedb892f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current keras workflow


Epoch 1/20
1/1 [==============================] - 12s 12s/step - loss: 0.6706 - accuracy: 0.5625
Epoch 2/20
1/1 [==============================] - 0s 431ms/step - loss: 1.0915 - accuracy: 0.4375
Epoch 3/20
1/1 [==============================] - 1s 941ms/step - loss: 0.5957 - accuracy: 0.6250
Epoch 4/20
1/1 [==============================] - 0s 376ms/step - loss: 0.4919 - accuracy: 0.7812
Epoch 5/20
1/1 [==============================] - 0s 398ms/step - loss: 0.3825 - accuracy: 0.8750
Epoch 6/20
1/1 [==============================] - 0s 389ms/step - loss: 0.4954 - accuracy: 0.7188
Epoch 7/20
1/1 [==============================] - 0s 399ms/step - loss: 0.4545 - accuracy: 0.8438
Epoch 8/20
1/1 [==============================] - 0s 425ms/step - loss: 0.2279 - accuracy: 1.0000
Epoch 9/20
1/1 [==============================] - 0s 236ms/step - loss: 0.2531 - accuracy: 0.9062
Epoch 10/20
1/1 [==============================] - 0s 246ms/step - loss: 0.2673 - accuracy: 0.9062
Epoch 11/20
1/1 [===

2022/07/04 18:05:04 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.8.2+zzzcolab20220527125636) contains a local version label (+zzzcolab20220527125636). MLflow logged a pip requirement for this package as 'tensorflow==2.8.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/07/04 18:05:10 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpt7pzykdu/model, flavor: keras), fall back to return ['tensorflow==2.8.2', 'keras==2.8.0']. Set logging level to DEBUG to see the full traceback.


In [ ]:
import mlflow

mlflow.keras.log_model(model, "tensorflow-model")

INFO:tensorflow:Assets written to: /tmp/tmpd6zmr_4j/model/data/model/assets


2022/07/04 18:05:27 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.8.2+zzzcolab20220527125636) contains a local version label (+zzzcolab20220527125636). MLflow logged a pip requirement for this package as 'tensorflow==2.8.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/07/04 18:05:33 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.8.2+zzzcolab20220527125636) contains a local version label (+zzzcolab20220527125636). MLflow logged a pip requirement for this package as 'tensorflow==2.8.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/07/04 18:05:33 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.7+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). MLf

ModelInfo(artifact_path='tensorflow-model', flavors={'keras': {'keras_module': 'tensorflow.keras', 'keras_version': '2.8.0', 'save_format': 'tf', 'data': 'data', 'code': None}, 'python_function': {'loader_module': 'mlflow.keras', 'python_version': '3.7.13', 'data': 'data', 'env': 'conda.yaml'}}, model_uri='runs:/8c76d321faa24dfd9d3c1d9a3e7d0d4d/tensorflow-model', model_uuid='a42e5a83ba4b43468175bfc7ecf58162', run_id='8c76d321faa24dfd9d3c1d9a3e7d0d4d', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-07-04 18:05:25.253667', mlflow_version='1.27.0')